In [1]:
import psycopg2

In [2]:
conn = psycopg2.connect(
    database = 'postgres_db',
    user = 'postgres_user',
    password = 'admin',
    host = 'client.tuple-mia.com',
    port = '5432',
    sslmode = 'require')

In [3]:
temp = conn.cursor()

In [4]:
import yaml
with open('/home/ubuntu/MappingBuffer_event.yml','r') as f:
    doc = yaml.load(f)

In [5]:
print doc['table_map']
print doc['column_map']

{'CUSTOMER_CONTACT': 'contact', 'EVENT_LOG': 'event', 'TRANSACTION_MASTER': 'trans', 'CUSTOMER_SECONDARY': 'temp_customer_secondary', 'EVENT_MASTER': 'event_master', 'CUSTOMER_MASTER': 'users'}
{'CUSTOMER_CONTACT': {'phone_number': 'phone', 'cust_id': 'id', 'firstname': 'first_name', 'facebook_id': 'facebook_id', 'email_id': 'email', 'lastname': 'last_name'}, 'EVENT_LOG': {'timestamp': 'timestamp', 'cust_id': 'from_id', 'product_id': 'to_id', 'action_type': 'actiontype'}, 'TRANSACTION_MASTER': {'cust_id': 'userid', 'product_id': 'product_id', 'revenue': 'revenue', 'timestamp': 'purchase_date', 'renewal': 'renewal', 'quantity': 'quantity'}, 'CUSTOMER_SECONDARY': {'cust_id': 'cust_id'}, 'EVENT_MASTER': {'action_category_id': None, 'action_category_name': None, 'action_type': 'actiontype', 'action_name': 'action_desc'}, 'CUSTOMER_MASTER': {'create_date': 'create_date', 'birthday': 'birthday', 'cust_id': 'id'}}


In [6]:
query = 'SELECT * FROM %s'%(doc['table_map']['EVENT_LOG'])

In [7]:
print query

SELECT * FROM event


In [8]:
temp.execute(query)

In [9]:
event = temp.fetchall()

In [10]:
len(event)

7163170

In [11]:
col_names = [desc[0] for desc in temp.description]

In [12]:
col_names

['to_id', 'timestamp', 'actiontype', 'from_id']

In [13]:
import pandas as pd

In [14]:
event_table = pd.DataFrame(event, columns=col_names)

In [15]:
event_table.head()

,to_id,timestamp,actiontype,from_id
0,255074,2016-11-19 00:52:13+00:00,1,6148771
1,304603,2016-11-20 08:03:32+00:00,1,6148771
2,368895,2016-11-15 09:28:35+00:00,1,6148771
3,376784,2016-11-15 09:43:29+00:00,1,6148771
4,462976,2016-11-15 09:44:27+00:00,1,6148771


In [16]:
event_table.rename(columns={doc['column_map']['EVENT_LOG']['cust_id']:'cust_id'}, inplace=True)
event_table.rename(columns={doc['column_map']['EVENT_LOG']['product_id']:'product_id'}, inplace=True)

In [17]:
event_table.head()

,product_id,timestamp,actiontype,cust_id
0,255074,2016-11-19 00:52:13+00:00,1,6148771
1,304603,2016-11-20 08:03:32+00:00,1,6148771
2,368895,2016-11-15 09:28:35+00:00,1,6148771
3,376784,2016-11-15 09:43:29+00:00,1,6148771
4,462976,2016-11-15 09:44:27+00:00,1,6148771


In [18]:
event_df = event_table[['cust_id', 'product_id']]

In [19]:
event_df.head()

,cust_id,product_id
0,6148771,255074
1,6148771,304603
2,6148771,368895
3,6148771,376784
4,6148771,462976


In [20]:
len(event_df.cust_id.unique())

6256

In [21]:
len(event_df.product_id.unique())

386251

In [22]:
grp_prod = event_df.groupby(['product_id']).count()

In [23]:
grp_prod.head()

,cust_id
product_id,
1071,1
1153,2
9516,2
9700,2
9782,1


In [24]:
grp_prod = grp_prod.reset_index()

In [25]:
grp_prod.head()

,product_id,cust_id
0,1071,1
1,1153,2
2,9516,2
3,9700,2
4,9782,1


In [26]:
grp_prod.rename(columns={'cust_id':'value'}, inplace=True)

In [27]:
grp_prod.head()

,product_id,value
0,1071,1
1,1153,2
2,9516,2
3,9700,2
4,9782,1


In [28]:
grp_prod['value'].max()

1770

In [29]:
grp_prod_cou = grp_prod.groupby(['value']).count()

In [30]:
grp_prod_cou.head()

,product_id
value,
1,120224
2,53167
3,31734
4,21858
5,16395


In [31]:
grp_prod_cou = grp_prod_cou.reset_index()

In [32]:
grp_prod_cou.rename(columns = {'value':'no_of_users'}, inplace=True)
grp_prod_cou.rename(columns = {'product_id':'product_count'}, inplace=True)

In [33]:
grp_prod_cou.head()

,no_of_users,product_count
0,1,120224
1,2,53167
2,3,31734
3,4,21858
4,5,16395


In [34]:
grp_prod_cou.tail()

,no_of_users,product_count
1115,1595,1
1116,1603,2
1117,1636,1
1118,1694,1
1119,1770,1


In [35]:
grp_prod_cou['no_of_users'].max()

1770

In [36]:
reversed_df = grp_prod_cou.sort_index(ascending = False)

In [37]:
reversed_df.head()

,no_of_users,product_count
1119,1770,1
1118,1694,1
1117,1636,1
1116,1603,2
1115,1595,1


In [38]:
reversed_df['product_per'] = ""

In [39]:
reversed_df['aggregated_per'] = ""

In [40]:
reversed_df.head()

,no_of_users,product_count,product_per,aggregated_per
1119,1770,1,,
1118,1694,1,,
1117,1636,1,,
1116,1603,2,,
1115,1595,1,,


In [41]:
reversed_df['no_of_users'].sum()

666725

In [42]:
reversed_df['product_per'] = reversed_df['product_count']/reversed_df['product_count'].sum() * 100

In [43]:
reversed_df.head()

,no_of_users,product_count,product_per,aggregated_per
1119,1770,1,0.000259,
1118,1694,1,0.000259,
1117,1636,1,0.000259,
1116,1603,2,0.000518,
1115,1595,1,0.000259,


In [44]:
reversed_df['aggregated_per'] = reversed_df.product_per.cumsum()

In [45]:
reversed_df.head()

,no_of_users,product_count,product_per,aggregated_per
1119,1770,1,0.000259,0.000259
1118,1694,1,0.000259,0.000518
1117,1636,1,0.000259,0.000777
1116,1603,2,0.000518,0.001294
1115,1595,1,0.000259,0.001553


In [46]:
prod_df = reversed_df

In [47]:
prod_df.head()

,no_of_users,product_count,product_per,aggregated_per
1119,1770,1,0.000259,0.000259
1118,1694,1,0.000259,0.000518
1117,1636,1,0.000259,0.000777
1116,1603,2,0.000518,0.001294
1115,1595,1,0.000259,0.001553


In [53]:
prod_df

,no_of_users,product_count,product_per,aggregated_per
1119,1770,1,0.000259,0.000259
1118,1694,1,0.000259,0.000518
1117,1636,1,0.000259,0.000777
1116,1603,2,0.000518,0.001294
1115,1595,1,0.000259,0.001553
1114,1593,1,0.000259,0.001812
1113,1556,1,0.000259,0.002071
1112,1543,1,0.000259,0.002330
1111,1539,1,0.000259,0.002589
1110,1536,1,0.000259,0.002848


In [ ]:
prod = grp_prod.loc[grp_prod['value'] > 1]